In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
folder_path = "/content/drive/MyDrive/ecommerce_cosmetic_shop/"

In [4]:
files = [
    "2019-Oct.csv",
    "2019-Nov.csv",
    "2019-Dec.csv",
    "2020-Jan.csv",
    "2020-Feb.csv"
]

In [5]:
dfs = []
for f in files:
    df = pd.read_csv(folder_path + f, usecols=[
        "event_time", "event_type", "user_id",
        "product_id", "category_code", "brand", "price"
    ])
    # Added a column for file/month (easier tracking)
    df["source_file"] = f
    dfs.append(df)

In [6]:
master_df = pd.concat(dfs, ignore_index=True)

In [7]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_code  object 
 4   brand          object 
 5   price          float64
 6   user_id        int64  
 7   source_file    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 1.2+ GB


In [8]:
master_df["event_time"] = pd.to_datetime(master_df["event_time"])
master_df["event_date"] = master_df["event_time"].dt.date
master_df["event_month"] = master_df["event_time"].dt.to_period("M")

/tmp/ipython-input-3578224325.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  master_df["event_month"] = master_df["event_time"].dt.to_period("M")


In [9]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 10 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     object             
 2   product_id     int64              
 3   category_code  object             
 4   brand          object             
 5   price          float64            
 6   user_id        int64              
 7   source_file    object             
 8   event_date     object             
 9   event_month    period[M]          
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(5), period[M](1)
memory usage: 1.5+ GB


In [10]:
master_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,source_file,event_date,event_month
0,2019-10-01 00:00:00+00:00,cart,5773203,NaN,runail,2.62,463240011,2019-Oct.csv,2019-10-01,2019-10
1,2019-10-01 00:00:03+00:00,cart,5773353,NaN,runail,2.62,463240011,2019-Oct.csv,2019-10-01,2019-10
2,2019-10-01 00:00:07+00:00,cart,5881589,NaN,lovely,13.48,429681830,2019-Oct.csv,2019-10-01,2019-10
3,2019-10-01 00:00:07+00:00,cart,5723490,NaN,runail,2.62,463240011,2019-Oct.csv,2019-10-01,2019-10
4,2019-10-01 00:00:15+00:00,cart,5881449,NaN,lovely,0.56,429681830,2019-Oct.csv,2019-10-01,2019-10


#**Funnel Analysis**

### 1. Computing the overall funnel (entire dataset)
  - **conversion rate**
  - **drop off rate**
  - **overall conversion rate**

In [11]:
views = master_df[master_df["event_type"] == "view"]["user_id"].nunique()
carts = master_df[master_df["event_type"] == "cart"]["user_id"].nunique()
purchases = master_df[master_df["event_type"] == "purchase"]["user_id"].nunique()

In [12]:
funnel_df = pd.DataFrame({
    "Stage": ["View", "Cart", "Purchase"],
    "Users": [views, carts, purchases]
})

In [13]:
funnel_df["Conversion %"] = [
    None,
    round((carts / views) * 100, 2),
    round((purchases / carts) * 100, 2)
]

funnel_df["Drop-off %"] = [
    None,
    round((1 - carts / views) * 100, 2),
    round((1 - purchases / carts) * 100, 2)
]

In [42]:
overall_conv = round((purchases / views) * 100, 2)
dict_ = {'metric':['overall_conv'], 'value':[overall_conv]}
kpi_df = pd.DataFrame(dict_)
kpi_df.to_csv("kpi_df.csv", index=False)

In [46]:
overall_cart_ab = 100 - round((purchases/carts)*100 ,2)
dict_1 = {'metric':['cart_abandonment_rate'], 'value':[overall_cart_ab]}
cart_ab_df = pd.DataFrame(dict_1)
cart_ab_df.to_csv('cart_ab_df.csv', index=False)

In [ ]:
funnel_df.to_csv("funnel_df.csv", index=False)

In [15]:
print("Overall Conversion (View → Purchase):", overall_conv, "%")
print("\nFunnel Summary Table:")
display(funnel_df)

Overall Conversion (View → Purchase): 6.92 %

Funnel Summary Table:


,Stage,Users,Conversion %,Drop-off %
0,View,1597754,NaN,NaN
1,Cart,398308,24.93,75.07
2,Purchase,110518,27.75,72.25


### 2. Computing Month-Over-Month Funnel
- Finding **conversion** and **drop off rates** at each stage for **each month**
- Finding **overall conversion rate** for **each month**

In [16]:
monthly_views = master_df[master_df["event_type"] == "view"].groupby("event_month")["user_id"].nunique()
monthly_carts = master_df[master_df["event_type"] == "cart"].groupby("event_month")["user_id"].nunique()
monthly_purchases = master_df[master_df["event_type"] == "purchase"].groupby("event_month")["user_id"].nunique()

In [17]:
monthly_funnel = pd.DataFrame({
    "Views": monthly_views,
    "Carts": monthly_carts,
    "Purchases": monthly_purchases
}).reset_index()

In [18]:
monthly_funnel["Cart Rate (%)"] = (monthly_funnel["Carts"] / monthly_funnel["Views"] * 100).round(2)
monthly_funnel["Purchase Rate (%)"] = (monthly_funnel["Purchases"] / monthly_funnel["Carts"] * 100).round(2)
monthly_funnel["Overall Conv (%)"] = (monthly_funnel["Purchases"] / monthly_funnel["Views"] * 100).round(2)

In [19]:
monthly_funnel["View→Cart Drop (%)"] = (100 - monthly_funnel["Cart Rate (%)"]).round(2)
monthly_funnel["Cart→Purchase Drop (%)"] = (100 - monthly_funnel["Purchase Rate (%)"]).round(2)

In [20]:
monthly_funnel = monthly_funnel.sort_values("event_month")
monthly_funnel.to_csv("monthly_funnel.csv", index=False)

In [21]:
print("\nMonth-over-Month Funnel Summary")
display(monthly_funnel)


Month-over-Month Funnel Summary


,event_month,Views,Carts,Purchases,Cart Rate (%),Purchase Rate (%),Overall Conv (%),View→Cart Drop (%),Cart→Purchase Drop (%)
0,2019-10,388331,133818,25762,34.46,19.25,6.63,65.54,80.75
1,2019-11,355643,95939,31524,26.98,32.86,8.86,73.02,67.14
2,2019-12,358212,83458,25613,23.30,30.69,7.15,76.70,69.31
3,2020-01,397775,92653,28220,23.29,30.46,7.09,76.71,69.54
4,2020-02,379246,89269,25759,23.54,28.86,6.79,76.46,71.14


### 3. Computing the impact of cart removes
- **Month over Month**
- Calculating **Net Carts** (Carts left after those which were removed)
- Computing **conversion** and **drop off** rates at each stage
- Computing **overall conversion rate**

In [22]:
monthly_views = master_df[master_df["event_type"] == "view"].groupby("event_month")["user_id"].nunique()
monthly_carts = master_df[master_df["event_type"] == "cart"].groupby("event_month")["user_id"].nunique()
monthly_removes = master_df[master_df["event_type"] == "remove_from_cart"].groupby("event_month")["user_id"].nunique()
monthly_purchases = master_df[master_df["event_type"] == "purchase"].groupby("event_month")["user_id"].nunique()

monthly_funnel_adj = pd.DataFrame({
    "Views": monthly_views,
    "Carts (Gross)": monthly_carts,
    "Removes": monthly_removes,
    "Purchases": monthly_purchases
}).reset_index()

In [23]:
monthly_funnel_adj["Carts (Net)"] = (monthly_funnel_adj["Carts (Gross)"] - monthly_funnel_adj["Removes"])

In [24]:
monthly_funnel_adj["Cart Rate (Net %)"] = (monthly_funnel_adj["Carts (Net)"] / monthly_funnel_adj["Views"] * 100).round(2)
monthly_funnel_adj["Purchase Rate (Net %)"] = (monthly_funnel_adj["Purchases"] / monthly_funnel_adj["Carts (Net)"] * 100).round(2)
monthly_funnel_adj["Overall Conv (Net %)"] = (monthly_funnel_adj["Purchases"] / monthly_funnel_adj["Views"] * 100).round(2)

monthly_funnel_adj["View→Cart Drop (Net %)"] = (100 - monthly_funnel_adj["Cart Rate (Net %)"]).round(2)
monthly_funnel_adj["Cart→Purchase Drop (Net %)"] = (100 - monthly_funnel_adj["Purchase Rate (Net %)"]).round(2)

In [25]:
monthly_funnel_adj = monthly_funnel_adj.sort_values("event_month")
monthly_funnel_adj.to_csv("monthly_funnel_adj.csv", index=False)

print("\nAdjusted Funnel with remove_from_cart considered")
display(monthly_funnel_adj)


Adjusted Funnel with remove_from_cart considered


,event_month,Views,Carts (Gross),Removes,Purchases,Carts (Net),Cart Rate (Net %),Purchase Rate (Net %),Overall Conv (Net %),View→Cart Drop (Net %),Cart→Purchase Drop (Net %)
0,2019-10,388331,133818,47090,25762,86728,22.33,29.70,6.63,77.67,70.30
1,2019-11,355643,95939,54628,31524,41311,11.62,76.31,8.86,88.38,23.69
2,2019-12,358212,83458,45217,25613,38241,10.68,66.98,7.15,89.32,33.02
3,2020-01,397775,92653,51107,28220,41546,10.44,67.92,7.09,89.56,32.08
4,2020-02,379246,89269,49053,25759,40216,10.60,64.05,6.79,89.40,35.95


### 4. Funnel segmentation by brand
- segmenting by **top 5 most frequently bought brands**
- **Month over Month**
- computing **conversion**, **drop off** and **overall conversion rate** for each brand

In [26]:
master_df['brand'].value_counts()

,count
brand,
runail,1528908
irisk,1033852
masura,861763
grattol,852591
bpw.style,434813
...,...
shifei,9
vl-gel,7
dessata,6


In [27]:
top_brands = (
    master_df[master_df['event_type'] == 'view']
    .groupby('brand')['user_id']
    .nunique()
    .sort_values(ascending=False)
    .head(5)
    .index
)

In [28]:
brand_df = master_df[master_df['brand'].isin(top_brands)]

In [29]:
funnel_by_brand = (
    brand_df.groupby(['brand', 'event_type'])['user_id']
    .nunique()
    .reset_index()
    .pivot(index='brand', columns='event_type', values='user_id')
    .fillna(0)
    .reset_index()
)

In [30]:
funnel_by_brand['Cart Rate (%)'] = (funnel_by_brand['cart'] / funnel_by_brand['view']) * 100
funnel_by_brand['Purchase Rate (%)'] = (funnel_by_brand['purchase'] / funnel_by_brand['cart']) * 100
funnel_by_brand['Overall Conv (%)'] = (funnel_by_brand['purchase'] / funnel_by_brand['view']) * 100

In [31]:
funnel_by_brand = funnel_by_brand[['brand', 'view', 'cart', 'purchase',
                                   'Cart Rate (%)', 'Purchase Rate (%)', 'Overall Conv (%)']]

In [32]:
funnel_by_brand = funnel_by_brand.sort_values('Overall Conv (%)', ascending=False)
funnel_by_brand.to_csv("funnel_by_brand.csv", index=False)
display(funnel_by_brand)

event_type,brand,view,cart,purchase,Cart Rate (%),Purchase Rate (%),Overall Conv (%)
2,irisk,153750,80769,30391,52.532683,37.627060,19.766504
4,runail,193942,92607,35411,47.749843,38.237930,18.258552
1,grattol,106093,49325,17150,46.492229,34.769387,16.165063
0,estel,92359,27330,9190,29.591052,33.626052,9.950303
3,kapous,92035,21724,7217,23.604064,33.221322,7.841582


## **Retention Cohort**

In [33]:
#finding all those customers who purchased in the entire 5 month dataset.
purchase_df = master_df[master_df['event_type']=='purchase']

months = ['2019-10','2019-11','2019-12','2020-01','2020-02']

retention = {}

'''
enumerate(months) will allow us to have values of both:
i = index of month (2019-10 having index = 0, 2019-11 having index = 1 and so on)
cohort_moth = month itself

loop 1:
each of the month above (in the months arr), becomes the cohort month one by one.
for eg: first, oct 2019 is the cohort month.
now, for this oct cohort, we get the set of all unique users which purchased in this cohort (cohort_users)
we also get the number of unique users in this cohort (cohort_size)
cohort_users = users who purchased in oct cohort.
for each month, the initial retention is kept to be 100 as we are going to find the retention cohort with respect to each month.

loop 2:
here Nov-Feb become the next months, if cohort month is Oct. If, say the next cohort month is Nov, the next months become Dec-Feb.
for each of these next months, set of unique users who purchased is computed and this set is called next_users.
now, next_users have all kind of users which purchased. it has new users of next month (any month from Nov-Feb) as well as returning users of the cohort month (here, it's Oct).
we want just the returning users from Oct (cohort_users) in all the next months. hence, we don't just directly compute the length, but with a condition:
cohort_users & next_users which gives us number only those users from next month which were present in the cohort month.
'''

for i, cohort_month in enumerate(months):
    # Users who purchased in the cohort month
    cohort_users = set(purchase_df[purchase_df['event_month'] == cohort_month]['user_id'])
    cohort_size = len(cohort_users)
    retention[cohort_month] = [100]  # Month 0 = 100% retention

    # For each subsequent month, finding how many of the original cohort purchased
    for next_month in months[i+1:]:
        next_users = set(purchase_df[purchase_df['event_month'] == next_month]['user_id'])
        retained = len(cohort_users & next_users)
        retention[cohort_month].append(round(retained / cohort_size * 100, 2))

In [40]:
retention_df = pd.DataFrame.from_dict(retention, orient='index')
retention_df.columns = ['Month 0', 'Month 1', 'Month 2', 'Month 3', 'Month 4']

retention_df = retention_df.reset_index().rename(columns={'index': 'Cohort Month'})

retention_df.to_csv("retention_df.csv", index=False)

print("Retention Matrix (%)")
display(retention_df)

Retention Matrix (%)


,Cohort Month,Month 0,Month 1,Month 2,Month 3,Month 4
0,2019-10,100,18.49,12.78,13.19,10.45
1,2019-11,100,13.76,14.63,11.61,NaN
2,2019-12,100,15.51,12.59,NaN,NaN
3,2020-01,100,15.97,NaN,NaN,NaN
4,2020-02,100,NaN,NaN,NaN,NaN
